In [5]:
import cv2
import numpy as np

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from utils import *

In [8]:
if __name__ == "__main__":
    
    dataset_path='../dataset/rgbd_dataset_freiburg1_xyz/'
    depth_paths=dataset_path+'depth.txt'
    dlist=data(depth_paths)

    rgb_paths=dataset_path+'rgb.txt'
    ilist=data(rgb_paths)

    
    

    for i in range(len(dlist)):

        frame=cv2.imread(dataset_path+ilist[i])
        depth=cv2.imread(dataset_path+dlist[i],-1)
        # print(frame)

        if frame is None:
            print("End of frame")
            break

        cv2.imshow("RGB",frame)
        cv2.imshow("Depth",depth)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


        
